In [1]:
!pip install -q dask_cuda torch torchtext skorch
!pip -q install dask[dataframe] --upgrade

# Hyperparameter optimization with Skorch


## Setup Dask

In [2]:
import torch
from dask_cuda import LocalCUDACluster
from distributed import Client

# if you have GPU(s), use dask_cuda to automatically make use of them in your dask cluster
if torch.cuda.is_available():
    cluster = LocalCUDACluster()
    client = Client(cluster)
else:
    client = Client(processes=False, threads_per_worker=4,
                    n_workers=1, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:38419 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.68 GB


In [3]:
# for reproducibility
# NB: enabling reproducibility can significantly slow down runtimes
reproducible = True
if reproducible:
    import random
    import numpy as np

    SEED = 42

    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

## Create Data

In [4]:
import torchtext
from torchtext import data
from torchtext import datasets
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# takes approx. 10 minutes to download data and embeddings (will be cached for re-use)

# set up fields
TEXT = data.Field(lower=True, batch_first=True, )
LABEL = data.Field(sequential=False, unk_token=None)

# make splits for data
train, test = datasets.IMDB.splits(TEXT, LABEL)

# work with 5k datapoints for faster iteration times
split_ratio = 5_000 / len(train)
train, discard = train.split(split_ratio=split_ratio)

split_ratio = 5_000 / len(test)
test, discard = test.split(split_ratio=split_ratio)

# will be used to initialize model embeddings layer
vocab = torchtext.vocab.GloVe(name='6B', dim=100)

# build the vocabulary
max_size = 25_000 # shorten for demonstrative purposes
TEXT.build_vocab(train, vectors=vocab, max_size=max_size)
LABEL.build_vocab(train)

# sadly I don't think we can use this 
# make iterator for splits
# train_iter, test_iter = data.BucketIterator.splits(
#     (train, test), batch_sizes=(32, 64), device='cpu')

In [7]:
# itos := index-to-string
# note the 2 extra tokens added for us: '<unk>', '<pad>'
TEXT.vocab.itos[:5]

['<unk>', '<pad>', 'the', 'a', 'and']

In [8]:
# stoi := string-to-index
# check on the meaning of these zeroes and ones
LABEL.vocab.stoi

defaultdict(None, {'neg': 0, 'pos': 1})

In [9]:
assert (len(TEXT.vocab.itos) == max_size + 2)

In [10]:
# peek at the data
print(train.examples[0].text)
print()
print(train.examples[0].label)

['it', 'aired', 'on', 'tv', 'yesterday,', 'so', 'i', 'decided', 'to', 'check', 'it', 'out.', 'this', 'was', 'one', 'of', 'the', 'last', 'bruce', 'timm/paul', 'dini', 'dtv', 'projects', 'related', 'to', 'their', 'old', '1992', 'batman', 'the', 'animated', 'series,', 'after', 'that', 'jeff', 'matsuda', 'came', 'along', 'and', 're-imagined', 'batman', 'with', 'his', 'new', 'the', 'batman', 'series,', 'but', 'anyway,', 'the', 'story', 'of', 'this', 'new', 'batman', 'movie', 'centers', 'around', 'the', 'appearance', 'of', 'a', 'new', 'vigilante', 'known', 'as', 'batwoman,', 'however', 'batman', 'feels', 'the', 'need', 'to', 'stop', 'her', 'because', 'of', 'her', 'extreme', 'methods,', 'and', 'also', 'in', 'the', 'meantime', 'take', 'down', 'the', 'pengiun', 'and', 'ruphert', 'thorn', 'who', 'both', 'are', 'secretly', 'working', 'with', 'carlton', "duquesne(who's", 'having', 'family', 'troubles)', 'and', 'another', 'villain(which', 'is', 'later', 'revealed', 'in', 'the', 'movie)', 'on', 'a',

In [11]:
# if you could use data.BucketIterator.splits

# sadly train_iter isn't actually an iter..
# peek at a batch of data
# batch = next(iter(train_iter))

# numericalized tokens
# print(batch.text)

# print(batch.label)

In [12]:
TEXT.process([train[0].text, train[1].text]).shape

torch.Size([2, 490])

In [13]:
# custom dataset class required to work with Skorch
class TorchDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        example = self.dataset.examples[idx]
        return example.text, example.label
    
    def __len__(self):
        return len(self.dataset)

In [14]:
# # custom dataset class required to work with Skorch
# class TorchDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset, TEXT, LABEL):
#         self.dataset = dataset
#         self.TEXT = TEXT
#         self.LABEL = LABEL
    
#     def __getitem__(self, idx):
#         example = self.dataset.examples[idx]
#         X = TEXT.numericalize([example.text]).squeeze() # get rid of "batch" dimension
#         y = LABEL.numericalize([example.label])
#         return X, y
    
#     def __len__(self):
#         return len(self.dataset)

In [15]:
ex = train.examples[0]

In [16]:
' '.join(ex.text)

"it aired on tv yesterday, so i decided to check it out. this was one of the last bruce timm/paul dini dtv projects related to their old 1992 batman the animated series, after that jeff matsuda came along and re-imagined batman with his new the batman series, but anyway, the story of this new batman movie centers around the appearance of a new vigilante known as batwoman, however batman feels the need to stop her because of her extreme methods, and also in the meantime take down the pengiun and ruphert thorn who both are secretly working with carlton duquesne(who's having family troubles) and another villain(which is later revealed in the movie) on a weapons smuggling operation,they also put a bounty on the batwoman. the question is: who is this mysterious batwoman and is it possible that they could be more then one? it's up to batman to solve this mystery and stop penguin's latest operation. for an animated movie, it has a fairly complex plot and a serious tone, which is good. another

In [17]:
train_dataset = TorchDataset(train)
test_dataset = TorchDataset(test)

In [18]:
print(train_dataset[0])

(['it', 'aired', 'on', 'tv', 'yesterday,', 'so', 'i', 'decided', 'to', 'check', 'it', 'out.', 'this', 'was', 'one', 'of', 'the', 'last', 'bruce', 'timm/paul', 'dini', 'dtv', 'projects', 'related', 'to', 'their', 'old', '1992', 'batman', 'the', 'animated', 'series,', 'after', 'that', 'jeff', 'matsuda', 'came', 'along', 'and', 're-imagined', 'batman', 'with', 'his', 'new', 'the', 'batman', 'series,', 'but', 'anyway,', 'the', 'story', 'of', 'this', 'new', 'batman', 'movie', 'centers', 'around', 'the', 'appearance', 'of', 'a', 'new', 'vigilante', 'known', 'as', 'batwoman,', 'however', 'batman', 'feels', 'the', 'need', 'to', 'stop', 'her', 'because', 'of', 'her', 'extreme', 'methods,', 'and', 'also', 'in', 'the', 'meantime', 'take', 'down', 'the', 'pengiun', 'and', 'ruphert', 'thorn', 'who', 'both', 'are', 'secretly', 'working', 'with', 'carlton', "duquesne(who's", 'having', 'family', 'troubles)', 'and', 'another', 'villain(which', 'is', 'later', 'revealed', 'in', 'the', 'movie)', 'on', 'a'

In [19]:
def pad_batch(batch, TEXT, LABEL):
    text, label = list(zip(*batch))
    # numericalized and padded text representation
    text_processed = TEXT.process(text)
    label_processed = LABEL.process(label)
    return text_processed, label_processed

from functools import partial

pad_batch_partial = partial(pad_batch, TEXT=TEXT, LABEL=LABEL)

In [20]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_batch_partial)

In [21]:
batch = next(iter(train_dataloader))

## Define your network

In [22]:
# was having trouble with when model was defined in the notebook
# Can't get attribute ‘CNN' on <module ‘__main__'
from model import CNN

In [23]:
# smoketest
model = CNN(pretrained_embeddings=TEXT.vocab.vectors).to(device)
gpu_batch = batch[0].to(device)
model_out = model(gpu_batch)
print(model_out)

tensor([[0.1738, 0.8262],
        [0.3462, 0.6538],
        [0.3317, 0.6683],
        [0.2430, 0.7570],
        [0.3659, 0.6341],
        [0.3846, 0.6154],
        [0.3757, 0.6243],
        [0.3713, 0.6287],
        [0.3153, 0.6847],
        [0.3656, 0.6344],
        [0.4096, 0.5904],
        [0.3088, 0.6912],
        [0.2832, 0.7168],
        [0.3461, 0.6539],
        [0.2674, 0.7326],
        [0.2894, 0.7106],
        [0.3850, 0.6150],
        [0.2820, 0.7180],
        [0.3537, 0.6463],
        [0.2389, 0.7611],
        [0.4709, 0.5291],
        [0.4199, 0.5801],
        [0.4443, 0.5557],
        [0.3591, 0.6409],
        [0.3270, 0.6730],
        [0.3097, 0.6903],
        [0.3241, 0.6759],
        [0.3594, 0.6406],
        [0.3175, 0.6825],
        [0.3447, 0.6553],
        [0.4566, 0.5434],
        [0.2943, 0.7057]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [24]:
del model

In [25]:
del gpu_batch

In [26]:
torch.cuda.empty_cache()

## Quick attempt at model training

In [27]:
import skorch
from skorch import NeuralNetClassifier 
import torch.optim as optim

In [28]:
import torch.nn as nn

In [29]:
# if you want to use a custom DataLoader, you must use NeuralNet
# also, not immediately obvious that for NeuralNet you are responsible for applying the log function
# whereas for NeuralNetClassifier, you are not

# NB: not ideal to be using softmax + log + NLLLoss
# see discussion: https://github.com/skorch-dev/skorch/issues/637
skorch_model = NeuralNetClassifier(
                CNN,
                device=device,
                max_epochs=2,
                lr=0.001,
                optimizer=optim.Adam,
                criterion=nn.NLLLoss,
                iterator_train=DataLoader,
                iterator_train__shuffle=True,
                iterator_train__batch_size=32,
                iterator_train__collate_fn=pad_batch_partial,
                iterator_train__num_workers=8,
                iterator_valid=DataLoader,
                iterator_valid__shuffle=False,
                iterator_valid__batch_size=64,
                iterator_valid__collate_fn=pad_batch_partial,
                iterator_valid__num_workers=8,
                train_split=skorch.dataset.CVSplit(.2), # NB: this witholds 20% of the training data for validation
                module__n_filters=100,
                module__filter_sizes=(2,3,4),
                module__dropout=0.2,
                module__pretrained_embeddings=TEXT.vocab.vectors,
                verbose=2)
# getting the following error when trying to compute accuracy
# ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets
#                 callbacks=callbacks)

In [30]:
skorch_model.fit(train_dataset, y=None)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.6295       0.7840        0.5172  12.0888
      2        0.4530       0.8170        0.4310  12.4245


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=CNN(
    (embedding): Embedding(25002, 100)
    (conv_0): Conv1d(1, 100, kernel_size=(2, 100), stride=(1,))
    (conv_1): Conv1d(1, 100, kernel_size=(3, 100), stride=(1,))
    (conv_2): Conv1d(1, 100, kernel_size=(4, 100), stride=(1,))
    (fc): Linear(in_features=300, out_features=2, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  ),
)

In [31]:
# this isn't working as expected
# skorch_model.score(test_dataset)
# TypeError: score() missing 1 required positional argument: 'y'
# skorch_model.score(test_dataset, y=None)
# ValueError: Expected array-like (array or non-string sequence), got None

In [32]:
# score manually
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False, collate_fn=pad_batch_partial, num_workers=8)

In [33]:
# sadly processing the entire test set twice just to score the model
processed_test_data = next(iter(test_dataloader))

In [34]:
# quick check on the test set accuracy
test_preds = skorch_model.predict(test_dataset)

In [35]:
test_labels = processed_test_data[1].numpy()

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(test_labels, test_preds)

0.8066

In [38]:
# random guessing would 50% accuracy so the model is indeed training
np.unique(test_labels, return_counts=True)

(array([0, 1]), array([2521, 2479]))

In [39]:
# NB: this has no effect on GPU memory usage. If I keyboard interrupt, the workers get
# restarted and memory usage goes down. Deleting these "handler" objects doesn't delete
# GPU memory references on the workers. 
del skorch_model

## Grid search with Hyperband

In [40]:
# try to store the raw text in dask arrays and handle preprocessing inside the network
# this isn't ideal because feature engineering really should be separate from modeling

In [41]:
# raw text
train_text = []
train_label = []
for e in train.examples:
    tokenized_text = e.text
    text_string = ' '.join(tokenized_text)
    train_text.append(text_string)
    label = e.label
    # label = LABEL.process([e.label]).numpy()
    train_label.append(label)

In [42]:
train_text[:1]

["it aired on tv yesterday, so i decided to check it out. this was one of the last bruce timm/paul dini dtv projects related to their old 1992 batman the animated series, after that jeff matsuda came along and re-imagined batman with his new the batman series, but anyway, the story of this new batman movie centers around the appearance of a new vigilante known as batwoman, however batman feels the need to stop her because of her extreme methods, and also in the meantime take down the pengiun and ruphert thorn who both are secretly working with carlton duquesne(who's having family troubles) and another villain(which is later revealed in the movie) on a weapons smuggling operation,they also put a bounty on the batwoman. the question is: who is this mysterious batwoman and is it possible that they could be more then one? it's up to batman to solve this mystery and stop penguin's latest operation. for an animated movie, it has a fairly complex plot and a serious tone, which is good. anothe

In [43]:
train_text = np.array(train_text)

In [44]:
train_text = np.expand_dims(train_text, axis=-1)

In [45]:
sample_batch = [train_text[0], train_text[10]]

In [46]:
sample_text_batch = [i[0] for i in sample_batch]

In [47]:
TEXT.process(sample_text_batch)

tensor([[    9, 12553,     0,  ...,     3,  7166,   369],
        [    3,  7166, 12553,  ...,     1,     1,     1]])

In [48]:
TEXT.process([train_text[0][0], train_text[1][0]]).shape

torch.Size([2, 2825])

In [49]:
isinstance(train_text[:10], np.ndarray)

True

In [50]:
# TEXT.process(train_text[0])

In [51]:
train_label = np.array(train_label)

In [52]:
# # this is a really unfortunate hack to make torchtext batching semantics work with skorch and dask
# # the downside here is that we're no longer padding to the longest sequence in the batch, rather
# # we're padding to the longest sequence in the *dataset*, which results in signifcantly more
# # computation and thus significantly more time to train a model
# # of course, you could set a max sequence length but that's not an ideal solution
# # another solution would be to create a different dataset object, but then you can't use torchtext,
# # which really is quite handy

# # train=True shuffles the data
# train_iter_skorch = torchtext.data.Iterator(train, batch_size=len(train), train=True, sort=False, device='cpu')
# test_iter_skorch = torchtext.data.Iterator(test, batch_size=len(test), train=False, sort=False, device='cpu')

# # takes some time to numericalize the whole dataset

# # also notice that skorch and dask expect numpy arrays, which isn't ideal since it ties you to the cpu.
# # meanwhile, projects like https://rapids.ai/ are moving toward all GPU computation, avoiding the cpu altogether.
# for batch in train_iter_skorch:
#     X_train = batch.text[0].numpy()
#     y_train = batch.label.numpy()

# for batch in test_iter_skorch:
#     X_test = batch.text[0].numpy()
#     y_test = batch.label.numpy()

In [53]:
# notice how awfully large the second dimension is
# X_train.shape

In [54]:
# https://ml.dask.org/hyper-parameter-search.html#hyperband-parameters-rule-of-thumb
EPOCHS = 2
NUM_TRAINING_EXAMPLES = len(train)*.8
n_examples = EPOCHS * NUM_TRAINING_EXAMPLES
n_params = 8

# it's not immediately obvious to beginners how all these parameters interact with each other
max_iter = n_params
chunk_size = n_examples // n_params

In [55]:
# suppose we want to set max_iter to be the commensurate with the number of examples required
# for the model converge (as cited in the documentation)

# it's a bit unclear how n_params relates to BOTH the number of data points required
# for the model to converge AND how many hyperparameters to try out (i.e. n_iter in RandomizedSearchCV)

In [56]:
import math

In [57]:
# choose chunk size so that the remainder is not a tiny number
print(f'Chunk size: {chunk_size}')
print(f'Total chunks: {math.ceil(len(train) / chunk_size)}')
last_chunk_size = len(train) % chunk_size
if last_chunk_size == 0: # i.e. chunk_size evenly divides X_train
    last_chunk_size = chunk_size
print(f'Last chunk size: {last_chunk_size}')

assert (len(train) % chunk_size > 10 or len(train) % chunk_size == 0), 'Choose another chunk size'

Chunk size: 1000.0
Total chunks: 5
Last chunk size: 1000.0


In [58]:
import dask.array as da

X = da.from_array(train_text, chunks=(chunk_size))
y = da.from_array(train_label, chunks=(chunk_size))

In [59]:
X

dask.array<array, shape=(5000, 1), dtype=<U10363, chunksize=(1000, 1), chunktype=numpy.ndarray>

In [60]:
# need new collate_fn due to the following error
# ValueError: Expected 2D array, got 1D array instead:
# array=['1' '1' '1' ... '1' '1' '1'].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [61]:
def pad_batch_hyperband(batch, TEXT, LABEL):
    text, label = list(zip(*batch))
    # unnecessary extra dimension
    text = [i[0] for i in text]
    # numericalized and padded text representation
    text_processed = TEXT.process(text)
    label_processed = LABEL.process(label)
    return text_processed, label_processed

pad_batch_hyperband_partial = partial(pad_batch_hyperband, TEXT=TEXT, LABEL=LABEL)

In [62]:
# reinitialize and set train_split=None to let hyperband handle validation set splitting
skorch_model = NeuralNetClassifier(
                CNN,
                device=device,
                lr=0.001,
                optimizer=optim.Adam,
                criterion=nn.NLLLoss,
                iterator_train=DataLoader,
                iterator_train__shuffle=True,
                iterator_train__batch_size=32,
                iterator_train__collate_fn=pad_batch_hyperband_partial,
                iterator_valid=DataLoader,
                iterator_valid__collate_fn=pad_batch_hyperband_partial,
                iterator_valid__shuffle=False,
                iterator_valid__batch_size=64,
                train_split=None, # let hyperband handle it
                module__n_filters=100,
                module__filter_sizes=(2, 3, 4),
                module__dropout=0.2,
                module__pretrained_embeddings=TEXT.vocab.vectors,
                # module__TEXT=TEXT,
                batch_size=32,
                verbose=2)

In [63]:
from scipy.stats import loguniform

In [64]:
# define parameter grid
params = {'module__filter_sizes': [(1, 2, 3), (2, 3, 4), (3, 4, 5)], 
          'module__n_filters': [25, 50, 100],
          'module__dropout': loguniform(1e-1, 3e-1),
          'batch_size': [32, 64],
         }

In [65]:
from dask_ml.model_selection import HyperbandSearchCV

In [66]:
search = HyperbandSearchCV(
    skorch_model,
    params,
    max_iter=max_iter,
    verbose=True,
    test_size=0.2 # validation size
)

In [67]:
search.metadata["partial_fit_calls"]

26

In [68]:
search.metadata['n_models']

5

In [69]:
import time

In [70]:
# to clear up any confusion, every time partial_fit is called, we're passing in chunk_size number of
# data points. Then skorch handles the batch size either by being set explicitly or as part of the param grid.

# to compare this grid search to number of epochs, we have 26 partial_fit calls * 10k data points = 260k examples
# with a training set size of 25k * .8 = 20k data points, this is 13 epochs!
# considering that it takes approximately 5 epochs to train a model, you would get through less than 3 sets of 
# hyperparameters if manually searching. Instead we'll search through ~5.

In [71]:
# I feel like the GPU is spending a lot of time waiting for the CPU to preprocess data
# I've been watching GPU power consumption during training and it's not as high as during
# skorch training, so I tried to increase the number of workers the dataloader uses and
# got the following error: AssertionError: daemonic processes are not allowed to have children

In [72]:
# seems like the validation set is getting preprocessed twice
# KeyError: tensor([0.])

# but then if you remove the collate_fn from iterator_valid__collate_fn=pad_batch_hyperband_partial
# then you get the following error
# TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <U10363

In [73]:
# in general, I need to better understand what Hyperband is working on
# what happens after "creating x models"?
# it seems to take a fair amount of time before the GPU starts using power (i.e. model training)
# and then it's unclear how much progress has been made on the search

In [74]:
# I think I'm getting yelled at for passing in text
# TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <U13704

# NB: this took ~3800 seconds on a single Nvidia GTX 980 Ti
# notice how the number of training datapoints relates to the chunk size and our test_size
# e.g. Validation set size: 2500 = 12500*.2, Train set size: 10000 = 12500 - 2500
start = time.time()
search.fit(X, y)
end = time.time()
duration = round(end - start, 2)
print(f'Time to complete grid search: {duration} seconds')

[CV, bracket=1] creating 3 models


/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  sk_validation.check_array(sample, *args, **kwargs)
/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  sk_validation.check_array(sample, *args, **kwargs)
/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that

[CV, bracket=0] creating 2 models


/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  sk_validation.check_array(sample, *args, **kwargs)
/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  sk_validation.check_array(sample, *args, **kwargs)
/opt/conda/lib/python3.7/site-packages/dask_ml/utils.py:175: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that

[CV, bracket=0] For training there are between 800 and 800 examples in each chunk
[CV, bracket=1] For training there are between 800 and 800 examples in each chunk


tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 849, in callback
    result_list.append(f.result())
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.7/site-packages/dask_ml/model_selection/_incremental.py", line 625, in _fit
    prefix=self.prefix,
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.7/site-packages/dask_ml/model_selection/_incremental.py", line 233, in _fit
    metas = yield client.gather(new_scores)
  File "/opt/conda/lib/python3.7/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/opt/conda/lib/python3.7/s

KeyError: tensor([0.])

In [75]:
# what's going on here?
# train_label should be text data and then get handled appropriately by pad_batch_hyperband
# instead, by the time the label makes it into pad_batch_hyperband, it is already a float tensor
# hence the lookup error. I was expecting a text string like 'pos'

In [84]:
train_label[:2]

array(['pos', 'pos'], dtype='<U3')

In [80]:
y[:2]

dask.array<getitem, shape=(2,), dtype=<U3, chunksize=(2,), chunktype=numpy.ndarray>

In [82]:
y.dtype

dtype('<U3')

In [81]:
# why isn't this responding and why is the GPU running?
y[:2].compute()

KeyboardInterrupt: 

## Integration

`HyperbandSearchCV` follows the Scikit-learn API and mirrors Scikit-learn's `RandomizedSearchCV`. This means that it "just works". All the Scikit-learn attributes and methods are available:

In [ ]:
search.best_score_

In [ ]:
search.best_estimator_

In [ ]:
import pandas as pd

In [ ]:
search.cv_results_

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)
cv_results.head()

In [ ]:
# issue with numpy converting array of tuples into a 2d array
search.cv_results_['param_module__filter_sizes'] = search.cv_results_['param_module__filter_sizes'].tolist()

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)
cv_results.head()

In [ ]:
search.score(X_test, y_test)

In [ ]:
search.predict(X_test)

In [ ]:
search.predict(X_test).compute()

It also has some other attributes.

In [ ]:
hist = pd.DataFrame(search.history_)
hist.head()

This illustrates the history after every `partial_fit` call. There's also an attributed `model_history_` that records the history for each model (it's a reorganization of `history_`).

## Learn more

This notebook covered basic usage `HyperbandSearchCV`. The following documentation and resources might be useful to learn more about `HyperbandSearchCV`, including some of the finer use cases:

* [A talk](https://www.youtube.com/watch?v=x67K9FiPFBQ) introducing `HyperbandSearchCV` to the SciPy 2019 audience and the [corresponding paper](https://conference.scipy.org/proceedings/scipy2019/pdfs/scott_sievert.pdf)
* [HyperbandSearchCV's documentation](https://ml.dask.org/modules/generated/dask_ml.model_selection.HyperbandSearchCV.html)

Performance comparisons can be found in the SciPy 2019 talk/paper.